# ĐỒ ÁN: Xử lý ngôn ngữ tự nhiên
Đinh Hoàng Linh Đan     - 19521309
Trần Nguyễn Quỳnh Anh   - 19521217  

In [1]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Dataset

In [3]:
sentences = open('C:\\Users\Admin\\CS221.M11-POS_TAGGING_VIETNAMESE\\Dataset\\dataset.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [4]:
print('Số lượng câu trong Dataset:', len(sentences))
#sentences[0:len(sentences)]

Số lượng câu trong Dataset: 40


# Tách từ bằng Longest Matching

In [5]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [6]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [7]:
def longest_matching(sentence, two, three):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in two:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in three:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in two:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [8]:
two_word = load_n_grams('Resource\\two_word.txt')
three_word = load_n_grams('Resource\\three_word.txt')

In [9]:
with open('Tokens\\Longest_Matching_Tokens.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in sentences:
        word_list = longest_matching(sentence, two_word, three_word)
        longest_matching_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
longest_matching_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [10]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 156


# Tách từ bằng VnCoreNLP

In [11]:
%pip install vncorenlp

You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [12]:
from vncorenlp import VnCoreNLP

In [13]:
client = VnCoreNLP(address='http://127.0.0.1', port=9001)

In [14]:
with open('Tokens\\vncore_tokens.txt', 'w', encoding='utf-8') as f:
    vncore_sentences = []
    for sentence in sentences:
        word_list = client.tokenize(sentence)[0]
        vncore_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
vncore_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [15]:
count_vncore_compounds = 0
for sentence in vncore_sentences:
    for word in sentence.split():
        if '_' in word: count_vncore_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP:', count_vncore_compounds)

Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP: 163


# Tách từ thủ công

In [16]:
with open('Tokens\manual_token.txt', 'r', encoding='utf-8') as f:
    manual_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n': 
            manual_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
manual_tokenize_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [17]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 146


# Đánh giá kết quả tách từ

In [18]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [19]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word: pred_tp += 1
                total_true += 1
    return {
        'Accuracy': total_true / total_words, 
        'Precision': pred_tp / (pred_tp + pred_fp),
        'Recall': pred_tp / count_manual_tokenize_compounds,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [20]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, vncore_sentences)
manual_evaluation = tokenize_evaluation(manual_tokenize_sentences, vncore_sentences)
pd.DataFrame(
    [longest_matching_evaluation, manual_evaluation], 
    index = ['Longest Matching', 'Manual']
).astype(object).T


,Longest Matching,Manual
Accuracy,0.958549,0.964222
Precision,0.974359,0.979452
Recall,1.041096,0.979452
True Positive,152,143
False Positive,4,3
Total True,555,539
Total Errors,34,32


# Tạo ngữ liệu

In [21]:
manual_tokens = open('Tokens\\manual_token.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(manual_tokens))
manual_tokens[0:5]

Số lượng từ: 631


['Tương_lai\n', 'là\n', 'hôm_nay\n', '.\n', '\n']

In [22]:
longest_matching_tokens = open('Tokens\\Longest_Matching_Tokens.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(longest_matching_tokens))
longest_matching_tokens[0:5]

Số lượng từ: 629


['Tương_lai\n', 'là\n', 'hôm_nay\n', '.\n', '\n']

# Gán nhãn bằng VnCoreNLP

In [23]:
with open('Dataset\\label.txt', 'w', encoding='utf-8') as f:
    for word in manual_tokens:
        word = word.replace('\n', '')
        
        if '_' not in word: tag = client.pos_tag(word)
        else: tag = client.pos_tag(word.replace('_', ' '))
        
        if tag == []: f.write('\n')
        else: f.write(f'{word}\t{tag[0][0][1]}\n')
    f.write('\n')

# Chia dữ liệu thành tập Train và Test

In [24]:
label = open('Dataset\\label.txt', encoding='utf-8').readlines()
new_line_idx = [i for i, item in enumerate(manual_tokens) if item == '\n']

In [25]:
with open('Dataset\\label_train.txt', 'w', encoding='utf-8') as f:
    for line in label[:new_line_idx[32]]: f.write(line)
    f.write('\n')
    
with open('Dataset\\label_test.txt', 'w', encoding='utf-8') as f:
    for line in label[new_line_idx[32] + 1:]: f.write(line)

In [26]:
with open('Dataset\\train_words.txt', 'w', encoding='utf-8') as f:
    for line in label[:new_line_idx[32]]: f.write(re.sub('\t.*', '', line))
    f.write('\n')        
    
with open('Dataset\\test_words.txt', 'w', encoding='utf-8') as f:
    for line in label[new_line_idx[32] + 1:]: f.write(re.sub('\t.*', '', line))

# Preprocess Data

In [64]:
def preprocess(vocabs_dict, path):
    data = []
    file = open(path, encoding='utf-8').readlines()
    
    for index, word in enumerate(file):
        if not (word.lower()).split():
            word = '--n--'
            data.append(word)
            continue
        elif (word.lower()).strip() not in vocabs_dict:
            word = '--unknow--'
            data.append(word)
            continue
        data.append(word.strip())
    return data

In [28]:
def plot_tag_counts(label):
    tags = [word_tag.split()[1] for word_tag in label if word_tag.split()]
    tag_counts = pd.DataFrame(tags)[0].value_counts()
    tag_counts.plot.bar(rot=0, width=0.7, legend=False, figsize=(15, 5))
    return pd.DataFrame(tag_counts).T.assign(Total=tag_counts.sum()) 

# Vocabs

In [29]:
vocabs = open('Resource\\vocabs.txt', encoding='utf-8').read().split('\n')
vocabs_dict = {}
index = 0

for word in sorted(vocabs): 
    if word not in vocabs_dict: 
        vocabs_dict[word] = index  
        index += 1

In [30]:
print('Số lượng từ vựng:', len(vocabs_dict.keys()))
count = 0

for key, value in vocabs_dict.items():
    print(f'{key}: {value}')
    count += 1
    if count > 20: break

Số lượng từ vựng: 54820
: 0
!: 1
": 2
#: 3
$: 4
%: 5
&: 6
': 7
'': 8
'40s: 9
'60s: 10
'70s: 11
'80s: 12
'86: 13
'90s: 14
'N: 15
'S: 16
'd: 17
'em: 18
'll: 19
'm: 20


# Tập Train

In [31]:
label_train = open('Dataset\\label_train.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập label_train:', len(label_train))
label_train[0:5]

Số lượng từ trong tập label_train: 520


['Tương_lai\tNp\n', 'là\tV\n', 'hôm_nay\tN\n', '.\tCH\n', '\n']

In [66]:
train_words = preprocess(vocabs_dict, 'Dataset\\train_words.txt')
print('Số lượng từ trong tập train_words:', len(train_words))
train_words[0:5]

Số lượng từ trong tập train_words: 520


['Tương_lai', 'là', 'hôm_nay', '.', '--n--']

In [67]:
print('Các từ không nằm trong vocabs', end=': ')
for word_tag, word in zip(label_train, train_words):
    if word == '--unknow--': print(word_tag.split()[0], end=', ')
#plot_tag_counts(label_train)

Các từ không nằm trong vocabs: hủy, 1/3, khỏe, Tùy, thư_giãn, 

# Tập Test

In [34]:
label_test = open('Dataset\\label_test.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập test:', len(label_test))
label_test[0:5]

Số lượng từ trong tập test: 112


['Nếu\tC\n', 'chúng_ta\tP\n', 'đang\tR\n', 'mệt\tA\n', ',\tCH\n']

In [91]:
test_words = preprocess(vocabs_dict, 'Dataset\\test_words.txt')
print('Số lượng từ trong tập test_words:', len(test_words))
test_words[0:20]

Số lượng từ trong tập test_words: 112


['Nếu',
 'chúng_ta',
 'đang',
 'mệt',
 ',',
 'cơ_thể',
 'sẽ',
 'tự',
 'bắt',
 'ta',
 'phải',
 'ngủ',
 ',',
 'ngay',
 'khi',
 'ta',
 'đang',
 'bước_đi',
 '.',
 '--n--']

In [70]:
print('Các từ không nằm trong vocabs', end=': ')
for word_tag, word in zip(label_test, test_words):
    if word == '--unknow--': print(word_tag.split()[0], end=', ')
#plot_tag_counts(label_test)

Các từ không nằm trong vocabs: gây_ra, 

# Tags

In [71]:
def seperate_word_tag(word_tag, vocabs_dict): 
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
    else:
        word, tag = word_tag.split()
        if word not in vocabs_dict: word = '--unknow--'
    return word, tag

In [38]:
def create_dictionaries(train_gold, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    for word_tag in train_gold:
        word, tag = seperate_word_tag(word_tag, vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts


In [39]:
transition_counts, emission_counts, tag_counts = create_dictionaries(label_train, vocabs_dict)
states = sorted(tag_counts.keys())
print('Số nhãn:', len(states))
print(states)


Số nhãn: 16
['--s--', 'A', 'C', 'CH', 'Cc', 'E', 'L', 'M', 'N', 'Nc', 'Np', 'P', 'R', 'T', 'V', 'X']


In [40]:
print("Transition examples: ")
for example in list(transition_counts.items())[:10]:
    print(example)

Transition examples: 
(('--s--', 'Np'), 6)
(('Np', 'V'), 2)
(('V', 'N'), 20)
(('N', 'CH'), 11)
(('CH', '--s--'), 33)
(('--s--', 'R'), 2)
(('R', 'V'), 25)
(('V', 'V'), 51)
(('N', 'Cc'), 5)
(('Cc', 'V'), 5)


In [41]:
print("Emission examples: ")
for example in list(emission_counts.items())[:10]:
    print (example)


Emission examples: 
(('Np', '--unk--'), 6)
(('V', 'là'), 8)
(('N', 'hôm_nay'), 1)
(('CH', '.'), 33)
(('--s--', '--n--'), 33)
(('R', '--unk--'), 2)
(('V', 'tách'), 1)
(('V', 'khỏi'), 1)
(('N', 'quá_khứ'), 1)
(('Cc', 'và'), 11)


# Training

In [72]:
def seperate_word_tag(word_tag, vocabs_dict): 
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
    else:
        word, tag = word_tag.split()
        if word not in vocabs_dict: word = '--unknow--'
    return word, tag

In [73]:
def create_dictionaries(train_gold, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    for word_tag in train_gold:
        word, tag = seperate_word_tag(word_tag, vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts

In [74]:
transition_counts, emission_counts, tag_counts = create_dictionaries(label_train, vocabs_dict)
states = sorted(tag_counts.keys())
print('Số nhãn:', len(states))
print(states)

Số nhãn: 16
['--s--', 'A', 'C', 'CH', 'Cc', 'E', 'L', 'M', 'N', 'Nc', 'Np', 'P', 'R', 'T', 'V', 'X']


# Testing

In [45]:
def predict_pos(words, gold, emission_counts, vocabs_dict, states):
    num_correct = 0
    all_words = set(emission_counts.keys())
    
    for word, gold_tuple in zip(words, gold): 
        gold_tuple_list = gold_tuple.split()
        if len(gold_tuple_list) != 2: continue
        else: true_label = gold_tuple_list[1]
    
        count_final = 0
        pos_final = ''
        if word not in vocabs_dict: continue
        
        for pos in states:
            if (pos, word) not in emission_counts: continue
            count = emission_counts[(pos, word)]
            
            if count > count_final:
                count_final = count
                pos_final = pos
                    
        if pos_final == true_label: num_correct += 1
    accuracy = num_correct / len(gold)
    return accuracy

In [46]:
accuracy = predict_pos(train_words, label_train, emission_counts, vocabs_dict, states)
print('Độ chính xác trên tập train:', accuracy)

Độ chính xác trên tập train: 0.8884615384615384


In [47]:
accuracy = predict_pos(test_words, label_test, emission_counts, vocabs_dict, states)
print('Độ chính xác trên tập test:', accuracy)


Độ chính xác trên tập test: 0.5446428571428571


# Hidden Markov

In [48]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags, num_tags))
    trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i], all_tags[j])
            if key in transition_counts: count = transition_counts[key]
                
            count_prev_tag = tag_counts[all_tags[i]]
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)
    return A


In [49]:
alpha = 0.001
for i in range(len(states)): tag_counts.pop(i, None)

m = len(states)
A = create_transition_matrix(alpha, tag_counts, transition_counts)
df = pd.DataFrame(
    A[0:16, 0:16], 
    index = states[0:16], 
    columns = states[0:16]
)
df.head()

,--s--,A,C,CH,Cc,E,L,M,N,Nc,Np,P,R,T,V,X
--s--,0.000030,0.000030,0.090895,0.000030,0.000030,0.121184,0.030319,0.090895,0.212049,0.000030,0.181760,0.212049,0.060607,0.000030,0.000030,0.000030
A,0.000021,0.212715,0.021291,0.170176,0.063829,0.106368,0.042560,0.000021,0.085099,0.000021,0.000021,0.021291,0.042560,0.000021,0.233984,0.000021
C,0.000077,0.000077,0.000077,0.000077,0.000077,0.000077,0.076905,0.000077,0.153734,0.000077,0.000077,0.153734,0.307391,0.000077,0.307391,0.000077
CH,0.767180,0.000023,0.023270,0.000023,0.000023,0.000023,0.000023,0.000023,0.023270,0.000023,0.000023,0.069765,0.046518,0.000023,0.069765,0.000023
Cc,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.000091,0.181645,0.000091,0.000091,0.000091,0.363199,0.000091,0.453976,0.000091


In [50]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocabs):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocabs)
    
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (all_tags[i], vocabs[j])
            if key in emission_counts.keys(): count = emission_counts[key]
                
            count_tag = tag_counts[all_tags[i]]
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)
    return B


In [51]:
test_words_lower = []
for w in test_words:
    test_words_lower.append(w.lower())

In [52]:
train_words_lower = []
for w in test_words:
    train_words_lower.append(w.lower())

In [53]:
cidx  = test_words_lower
rvals = sorted(tag_counts.keys())
cols = [vocabs_dict[word] for word in cidx]
rows = [states.index(tag) for tag in rvals]

for i in range(len(states)): tag_counts.pop(i, None)
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocabs_dict))

df = pd.DataFrame(B[np.ix_(rows, cols)], index=rvals, columns=cidx)
df.head()


,nếu,chúng_ta,đang,mệt,",",cơ_thể,sẽ,tự,bắt,ta,...,đùng_đùng,hay,giữa,bom,đạn,hiểm_nguy,của,chiến_tranh,.,--n--
--s--,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,...,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.375780
A,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
C,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,...,0.000015,0.014760,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015
CH,0.000010,0.000010,0.000010,0.000010,0.102239,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.337365,0.000010
Cc,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,...,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015


In [54]:
A = np.array([sublist[1:].tolist() for sublist in A])
B = B[1:]


In [55]:
def viterbi_initialize(states, tag_counts, A, B, corpus, vocabs_dict):
    num_tags = len(tag_counts)
    s_idx = states.index('--s--')
    
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    for i in range(num_tags):
        if A[s_idx, i - 1] == 0: best_probs[i, 0] = float('-inf')
        else: 
            index = vocabs_dict[corpus[0]]
            best_probs[i, 0] = math.log(A[s_idx, i - 1]) + math.log(B[i - 1, index])
    return best_probs, best_paths


In [56]:
best_probs_train, best_paths_train = viterbi_initialize(states, tag_counts, A, B, train_words_lower, vocabs_dict)
best_probs_test, best_paths_test = viterbi_initialize(states, tag_counts, A, B, test_words_lower, vocabs_dict)

In [57]:
def viterbi_forward(A, B, corpus, best_probs, best_paths, vocabs_dict):
    num_tags = best_probs.shape[0] 
    
    for i in range(1, len(corpus)): 
        if i % 5000 == 0: print(f'Processed {i} words...')
            
        for j in range(num_tags):
            best_prob_i = float('-inf')
            best_path_i = None
            
            for k in range(num_tags):
                index = vocabs_dict[corpus[i]]
                prob = best_probs[k, i - 1] + math.log(A[k, j - 1]) + math.log(B[j - 1, index])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
                    
            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i
            
    return best_probs, best_paths



In [58]:
best_probs_train, best_paths_train = viterbi_forward(A, B, train_words_lower, best_probs_train, best_paths_train, vocabs_dict)
print('best_probs_train[0, 1]:', best_probs_train[0, 1]) 
print('best_paths_train[0, 4]:', best_paths_train[0, 4])

best_probs_train[0, 1]: -28.732564428426628
best_paths_train[0, 4]: 8


In [59]:
best_probs_test, best_paths_test = viterbi_forward(A, B, test_words_lower, best_probs_test, best_paths_test, vocabs_dict)
print('best_probs_test[0, 1]:', best_probs_test[0, 1]) 
print('best_paths_test[0, 4]:', best_paths_test[0, 4])

best_probs_test[0, 1]: -28.732564428426628
best_paths_test[0, 4]: 8


In [60]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    m = best_paths.shape[1] 
    z = [None] * m
    pred = [None] * m
    
    best_prob_for_last_word = float('-inf')
    num_tags = best_probs.shape[0]
    
    for k in range(num_tags):
        if best_probs[k, m - 1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, m - 1]
            z[m - 1] = k
            
    pred[m - 1] = states[z[m - 1]]
    for i in range(m - 1, -1, -1):
        z[i - 1] = best_paths[z[i], i]
        pred[i - 1] = states[z[i - 1]]
    return pred



In [75]:

train_pred = viterbi_backward(best_probs_train, best_paths_train, train_words, states)
test_pred = viterbi_backward(best_probs_test, best_paths_test, test_words, states)
m = len(test_pred)

print('Dự đoán cho test_pred[0:15]:')
print(test_words[0:15])
print(test_pred[0:15])

Dự đoán cho test_pred[0:15]:
['Nếu', 'chúng_ta', 'đang', 'mệt', ',', 'cơ_thể', 'sẽ', 'tự', 'bắt', 'ta', 'phải', 'ngủ', ',', 'ngay', 'khi']
['E', 'P', 'A', 'A', 'CH', 'N', 'R', 'V', 'E', 'P', 'V', 'V', 'CH', 'C', 'N']


In [62]:
for word, tag in zip(test_words, test_pred):
    if word == '--n--': print()
    else: print(f'{word}/{tag}', end=' ')


Nếu/E chúng_ta/P đang/A mệt/A ,/CH cơ_thể/N sẽ/R tự/V bắt/E ta/P phải/V ngủ/V ,/CH ngay/C khi/N ta/P đang/A bước_đi/A ./CH 
Khi/N buồn_ngủ/R là/V tự_nhiên/V phải/V ngủ/V ,/CH dù/C ý_chí/V của/E chúng_ta/P có/V mạnh_mẽ/A ./CH 
Hãy/R làm/V cho/E cơ_thể/N bạn/N mệt/R đến/V mức/C không_thể/R thức/V được/V nữa/A ./CH 
Lo_lắng/V về/E chứng/V mất_ngủ/V thường/R --unk--/V những/L hậu_quả/V còn/C tệ_hại/P hơn/A bản_thân/N căn_bệnh/N này/P ./CH 
Nhớ/A rằng/Cc chưa/R có/V ai/P chết/A vì/Cc thiếu/R ngủ/V ./CH 
Chúng_ta/P có_thể/R nhịn/V ăn/V ,/CH nhịn/P uống/A lâu/A hơn/A là/V nhịn/V ngủ/V ./CH 
Khi/N hoàn_toàn/A kiệt_sức/V ,/CH con_người/C vẫn/R có_thể/R ngủ/V được/V giữa/V trời/M sấm_sét/E đùng_đùng/P hay/C giữa/R bom/V đạn/V hiểm_nguy/M của/E chiến_tranh/N ./CH 


# Đánh giá

In [92]:
from sklearn.metrics import classification_report
def report(pred, gold):
    y_pred = []
    y_true = []

    for prediction, word_tag in zip(pred, gold):
        word_tag_tuple = word_tag.split()
        if len(word_tag_tuple) != 2: continue 

        word, tag = word_tag_tuple
        y_pred.append(prediction)
        y_true.append(tag)
        
    print(classification_report(y_pred, y_true))
    return y_pred, y_true


In [93]:
print('Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập test:\n')
y_pred, y_true_test = report(test_pred, label_test)

Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập test:

              precision    recall  f1-score   support

           A       0.50      0.46      0.48        13
           C       0.60      0.50      0.55         6
          CH       1.00      1.00      1.00        12
          Cc       0.00      0.00      0.00         2
           E       0.80      0.57      0.67         7
           L       1.00      1.00      1.00         1
           M       0.00      0.00      0.00         2
           N       0.45      1.00      0.62         9
          Np       0.00      0.00      0.00         0
           P       0.88      0.70      0.78        10
           R       0.80      0.67      0.73        12
           T       0.00      0.00      0.00         0
           V       0.79      0.74      0.77        31

    accuracy                           0.70       105
   macro avg       0.52      0.51      0.51       105
weighted avg       0.72      0.70      0.70       105

